# Projet : Systeme de recommandation d'images

## Presentation

Dans ce projet, vous allez construire un **systeme de recommandation d'images** qui suggere des images aux utilisateurs en fonction de leurs preferences. Ce projet met en pratique toutes les competences acquises lors des seances pratiques : analyse de donnees, visualisation, regroupement, classification et apprentissage automatique.

**Duree** : 3 seances pratiques
**Taille de l'equipe** : 2-3 etudiants
**Livrables** :
1. Un notebook Jupyter (`Nom1_Nom2_[Nom3].ipynb`)
2. Un rapport de synthese de 4 pages (PDF)

---

## Objectifs d'apprentissage

En completant ce projet, vous serez capable de :
- Automatiser la collecte de donnees a partir de sources web
- Extraire et traiter les metadonnees d'images
- Appliquer des algorithmes de regroupement pour analyser les caracteristiques des images
- Construire des profils de preferences utilisateur
- Implementer un algorithme de recommandation
- Visualiser efficacement les donnees
- Ecrire des tests complets pour votre systeme

---

## Architecture du projet

Le systeme est compose de 7 taches interconnectees :

```
┌─────────────────────────────────────────────────────────────────┐
│               SYSTEME DE RECOMMANDATION D'IMAGES                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 1. Collecte  │───▶│ 2. Etiquetage│───▶│ 3. Analyse   │       │
│  │ de donnees   │    │ & Annotation │    │ de donnees   │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────────────────────────────────────────────┐       │
│  │          Fichiers JSON (Stockage des metadonnees)     │       │
│  └──────────────────────────────────────────────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 4. Visuali-  │    │ 5. Systeme de│    │ 6. Tests     │       │
│  │ sation       │    │ recommandat. │    │              │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│                             │                                    │
│                             ▼                                    │
│                    ┌──────────────┐                              │
│                    │ 7. Rapport   │                              │
│                    │ de synthese  │                              │
│                    └──────────────┘                              │
└─────────────────────────────────────────────────────────────────┘
```

![Architecture](../../images/Project-Architecture.png "Architecture")

---

## Projet partie 1

---

### Tache 1 : Collecte de donnees

#### Objectif
Collecter au moins **100 images sous licence ouverte** avec leurs metadonnees.

#### Ce que vous devez faire

1. **Creer une structure de dossiers** :
   ```
   projet/
   ├── images/           # Images telechargees
   ├── data/             # Fichiers de metadonnees JSON
   └── projet.ipynb      # Votre notebook
   ```

2. **Trouver des sources d'images** (une ou plusieurs) :
   - [Wikimedia Commons](https://commons.wikimedia.org/) - Utilisez des requetes SPARQL (comme dans le TP 1)
   - [Unsplash API](https://unsplash.com/developers) - API gratuite pour des images de haute qualite
   - [Pexels API](https://www.pexels.com/api/) - Photos libres de droits
   - [Flickr API](https://www.flickr.com/services/api/) - Images Creative Commons

3. **Telecharger les images par programme** en utilisant les techniques du TP 1, Exercice 6

4. **Extraire et sauvegarder les metadonnees** de chaque image :
   - Nom du fichier image
   - Dimensions de l'image (largeur, hauteur)
   - Format du fichier (.jpg, .png, etc.)
   - Taille du fichier (en Ko)
   - URL source
   - Informations de licence
   - Donnees EXIF (si disponibles) : modele d'appareil photo, date de prise de vue, etc.

#### Resultat attendu
- Dossier `images/` contenant 100+ images
- `data/images_metadata.json` contenant les metadonnees de toutes les images

#### Conseils
- Utilisez `PIL` pour obtenir les dimensions de l'image
- Utilisez `os.path.getsize()` pour obtenir la taille du fichier
- Utilisez l'extraction EXIF (voir TP 2, Exercice 2)
- Stockez les metadonnees sous forme de liste de dictionnaires au format JSON

---

## Tâche 1 — Collecte de données : téléchargement des images
On récupère la liste des photos depuis l'API `jmail.world`, puis on télécharge les 100 premières dans `images/`.

In [7]:
import requests, time
from pathlib import Path

# ── Config ─────────────────────────────────────────────────────
LIMIT = 100
API_URL = "https://jmail.world/api/photos?newOnly=false"
CDN_BASE = "https://assets.getkino.com/photos-deboned"
IMAGES_DIR = Path("images")
IMAGES_DIR.mkdir(parents=True, exist_ok=True)

# ── 1. Récupérer la liste des photos depuis l'API ─────────────
session = requests.Session()
resp = session.get(API_URL, timeout=30)
resp.raise_for_status()
all_photos = resp.json()["photos"]

# Filtrer : garder seulement les images AVEC source_url (celles-ci sont dispos)
photos = [p for p in all_photos if p.get("source_url")][:LIMIT]
print(f"API : {len(all_photos)} photos total, {len(photos)} disponibles sur CDN")

# ── 2. Télécharger chaque image dans images/ ──────────────────
downloaded = 0
for i, photo in enumerate(photos, 1):
    filename = photo["original_filename"]
    dest = IMAGES_DIR / filename
    if dest.exists():
        downloaded += 1
        continue
    url = f"{CDN_BASE}/{filename}"
    try:
        r = session.get(url, stream=True, timeout=60)
        r.raise_for_status()
        with open(dest, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
        downloaded += 1
    except Exception as e:
        print(f"  ✗ {filename}: {e}")
    if i % 20 == 0:
        print(f"  [{i}/{len(photos)}]…")
    time.sleep(0.05)

print(f"\n✓ {downloaded}/{len(photos)} images dans {IMAGES_DIR}/")

API : 18316 photos total, 100 disponibles sur CDN
  [20/100]…
  [40/100]…
  [60/100]…
  [80/100]…
  [100/100]…

✓ 100/100 images dans images/
